**Run the following two cells before you begin.**

In [1]:
%autosave 10

Autosaving every 10 seconds


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
import os
os.chdir("D:\DataScience\Technocolabs\miniproject_2")

______________________________________________________________________
**First, import your data set and define the sigmoid function.**
<details>
    <summary>Hint:</summary>
    The definition of the sigmoid is $f(x) = \frac{1}{1 + e^{-X}}$.
</details>

In [4]:
# Import the data set
df =pd.read_csv("cleaned_data.csv")
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month,EDUCATION_CAT,graduate school,high school,others,university
0,798fc410-45c1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,1,university,0,0,0,1
1,8a8c8f3b-8eb4,120000,2,2,2,26,-1,2,0,0,...,1000,1000,0,2000,1,university,0,0,0,1
2,85698822-43f5,90000,2,2,2,34,0,0,0,0,...,1000,1000,1000,5000,0,university,0,0,0,1
3,0737c11b-be42,50000,2,2,1,37,0,0,0,0,...,1200,1100,1069,1000,0,university,0,0,0,1
4,3b7f77cc-dbc0,50000,1,2,1,57,-1,0,-1,0,...,10000,9000,689,679,0,university,0,0,0,1


In [5]:
# Define the sigmoid function
def sigmoidfun(x):
    return (1/(1+np.exp(-x)))


**Now, create a train/test split (80/20) with `PAY_1` and `LIMIT_BAL` as features and `default payment next month` as values. Use a random state of 24.**

In [6]:
# Create a train/test split
X = df[["PAY_1","LIMIT_BAL"]]
y = df[["default payment next month"]]
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=24,test_size=0.2)

______________________________________________________________________
**Next, import LogisticRegression, with the default options, but set the solver to `'liblinear'`.**

In [7]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver="liblinear")

______________________________________________________________________
**Now, train on the training data and obtain predicted classes, as well as class probabilities, using the testing data.**

In [8]:
# Fit the logistic regression model on training data
classifier.fit(X_train,y_train)

D:\softwares\Anaconda\program files\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [9]:
# Make predictions using `.predict()`
y_pred = classifier.predict(X_test)

In [10]:
# Find class probabilities using `.predict_proba()`
y_pre_proba = classifier.predict_proba(X_test)

______________________________________________________________________
**Then, pull out the coefficients and intercept from the trained model and manually calculate predicted probabilities. You'll need to add a column of 1s to your features, to multiply by the intercept.**

In [11]:
# Add column of 1s to features
ones = np.ones(len(df))
data = pd.DataFrame({"Ones":ones,"LIMIT_BAL":df.LIMIT_BAL,"PAY_1":df.PAY_1})
X_train_1,X_test_1,y_train_1,y_test_1 = train_test_split(data,y,random_state=24,test_size=0.2)

In [12]:
# Get coefficients and intercepts from trained model
coeff = classifier.coef_
intercept = classifier.intercept_

In [13]:
print(coeff)
print(intercept)

[[ 8.27451187e-11 -6.80876727e-06]]
[-6.57647457e-11]


In [14]:
# Manually calculate predicted probabilities
m1 = coeff[0][0]
m2 = coeff[0][1]
a = X_test_1.iloc[:,1]
b = X_test_1.iloc[:,2]
c = X_test_1.iloc[:,0]
z = m1*a + m2*b + c*intercept 
man_pred_prob = sigmoidfun(z)
all(man_pred_prob) == all(y_pre_proba[:,1])

True

______________________________________________________________________
**Next, using a threshold of `0.5`, manually calculate predicted classes. Compare this to the class predictions output by scikit-learn.**

In [15]:
# Manually calculate predicted classes
threshold = 0.5
man_pred = []
for i in man_pred_prob:
    if i >= 0.5:
        man_pred.append(0)
    else:
        man_pred.append(1)            

In [16]:
# Compare to scikit-learn's predicted classes
all(man_pred)==all(y_pred)

True

______________________________________________________________________
**Finally, calculate ROC AUC using both scikit-learn's predicted probabilities, and your manually predicted probabilities, and compare.**

In [17]:
# Use scikit-learn's predicted probabilities to calculate ROC AUC
from sklearn.metrics import roc_auc_score
roc_auc = roc_auc_score(y_test,y_pre_proba[:,1])
roc_auc

0.627207450280691

In [18]:
# Use manually calculated predicted probabilities to calculate ROC AUC
from sklearn.metrics import roc_auc_score
roc_auc = roc_auc_score(y_test_1,man_pred)
roc_auc

0.6370642760781583